check python version

In [1]:
!python -V

Python 3.7.13


install packages

In [2]:
!pip install bs4 lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


connect to drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


import packages

In [4]:
import lxml
from bs4 import BeautifulSoup as bs

open dataset and make b4s object of it

In [5]:
content = ""

with open("/content/drive/MyDrive/Hospital_log.xes", "r") as file:
  lines = file.readlines()
  content = "".join(lines)

dataset = bs(content, "lxml")

in case we want see data structure of an event

In [6]:
# traces = dataset.find_all("trace")

# traces[0].find_all("event")[0]

extract trace and events

In [7]:
transitions = []
archs = []
places = []
candid_places = []
start_transitions = []
end_transitions = []
traces = []

from datetime import datetime

for trace in dataset.find_all("trace"):
  events = []
  nameevents = []
  for event in trace.find_all("event"):
    name = event.find(attrs={"key": "concept:name"})['value']
    time = event.find(attrs={"key": "time:timestamp"})['value']
    if name not in transitions :
      transitions.append(name)

    if name not in nameevents :
      events.append({"name": name, "time": datetime.fromisoformat(time)})
      nameevents.append(name)
  
  events.sort(key=lambda item : item['time'])
  if events[0]['name'] not in start_transitions :
    start_transitions.append(events[0]['name'])
  
  if events[-1]['name'] not in end_transitions :
    end_transitions.append(events[-1]['name'])

  traces.append([e['name'] for e in events])

print('len transitions:{}'.format(len(transitions)))
print('len traces:{}'.format(len(traces)))
traces[0]

len transitions:624
len traces:1143


['1e consult poliklinisch',
 'administratief tarief       - eerste pol',
 'verlosk.-gynaec. korte kaart kosten-out',
 'echografie  - genitalia interna',
 'simulator - gebruik voor aanvang megavol',
 'behandeltijd - eenheid t3 - megavolt',
 'teletherapie - megavolt fotonen bestrali',
 'aanname laboratoriumonderzoek',
 'ureum',
 'hemoglobine foto-elektrisch',
 'creatinine',
 'natrium vlamfotometrisch',
 'kalium potentiometrisch',
 'leukocyten tellen elektronisch',
 'trombocyten tellen - elektronisch',
 'ordertarief',
 'ligdagen - alle spec.beh.kinderg.-reval.',
 'sediment - spoed',
 'bacteriologisch onderzoek met kweek -nie',
 'resistentiebepalingen - 5 bepalingen',
 'hepatitis-b surface antigeen confirmatie',
 'urine onderzoek - kwalitatief',
 '190021 klinische opname a002',
 '190205 klasse 3b        a205',
 'sgot - asat kinetisch',
 'sgpt - alat kinetisch',
 'melkzuurdehydrogenase -ldh- kinetisch',
 'differentiele telling automatisch',
 'trombotest',
 'crp c-reactief proteine',
 'brach

make footprint matrix

In [8]:
footprint = dict()

FOLLOW_REL = '>>'
CASUAL_REL = '->'
CONCUR_REL = '||'
INDEPC_REL = '##'

for x in transitions:
  footprint[x] = dict()
  for y in transitions:
    footprint[x][y] = INDEPC_REL

for t in traces:
  for i in range(len(t)-1):
    footprint[t[i]][t[i+1]] = FOLLOW_REL

for x in transitions:
  for y in transitions:
    if footprint[x][y] == FOLLOW_REL:
      if footprint[y][x] == FOLLOW_REL:
        footprint[x][y] = CONCUR_REL
      else:
        footprint[x][y] = CASUAL_REL


generate places

In [9]:
# first generations of places
for x in transitions:
  for y in transitions:
    if footprint[x][y] == CASUAL_REL and footprint[x][x] == INDEPC_REL and footprint[y][y] == INDEPC_REL:
      candid_places.append({"inps": {x,}, "outs":{y,}})

while True:
  nplaces = []
  for p1 in candid_places:
    for p2 in candid_places:
      cnd_cnt = False
      diffrent_inps = p1["inps"] ^ p2["inps"]
      diffrent_outs = p1["outs"] ^ p2["outs"]

      # equal place reached
      if diffrent_inps == {} and diffrent_outs == {}:
        continue

      # mergable place
      if diffrent_inps == {} or diffrent_outs == {}:

        ins = p1['inps']
        ous = p1['outs']

        for i in p2['inps']:
          if i not in ins:
            ins.append(i)

        for o in p2['outs']:
          if o not in ous:
            ous.append(o)

        p3 = {"inps": ins, "outs": ous}
        # print(p3)

        for i in p3['inps']:
          for o in p3['inps']:
            if i != o :
              if footprint[i][o] != INDEPC_REL:
                # ignore p3
                cnd_cnt = True

        for i in p3['outs']:
          for o in p3['outs']:
            if i != o :
              if footprint[i][o] != INDEPC_REL:
                # ignore p3
                cnd_cnt = True

        for i in p3['inps']:
          for o in p3['outs']:
            if footprint[i][o] != CASUAL_REL:
              # ignore p3
              cnd_cnt = True

        if cnd_cnt : continue

        nplaces.append(p3)
  
  # add generated places to places for next itteration
  for p in nplaces: 
    candid_places.append(p)

  if len(nplaces) == 0: 
    # merge not happend, exit loop
    break

candid_places

[{'inps': {'1e consult poliklinisch'},
  'outs': {'simulator - gebruik voor aanvang megavol'}},
 {'inps': {'1e consult poliklinisch'},
  'outs': {'behandeltijd - eenheid t3 - megavolt'}},
 {'inps': {'1e consult poliklinisch'},
  'outs': {'aanname laboratoriumonderzoek'}},
 {'inps': {'1e consult poliklinisch'}, 'outs': {'ureum'}},
 {'inps': {'1e consult poliklinisch'}, 'outs': {'natrium vlamfotometrisch'}},
 {'inps': {'1e consult poliklinisch'},
  'outs': {'ligdagen - alle spec.beh.kinderg.-reval.'}},
 {'inps': {'1e consult poliklinisch'},
  'outs': {'190021 klinische opname a002'}},
 {'inps': {'1e consult poliklinisch'},
  'outs': {'190205 klasse 3b        a205'}},
 {'inps': {'1e consult poliklinisch'}, 'outs': {'creatinine - spoed'}},
 {'inps': {'1e consult poliklinisch'},
  'outs': {'interc.consult klinisch  anesthesie'}},
 {'inps': {'1e consult poliklinisch'}, 'outs': {'sediment'}},
 {'inps': {'1e consult poliklinisch'}, 'outs': {'bilirubine - totaal'}},
 {'inps': {'1e consult polik

keep only maximal places

In [10]:
for p in candid_places:
  continue_cnd = False
  for chk in candid_places:
    # dont check with itself
    if p['inps'] != chk['inps'] or p['outs'] != chk['outs']:
      if p['inps'].issubset(chk['inps']) and p['outs'].issubset(chk['outs']):
        continue_cnd = True

  if continue_cnd:
    continue

  places.append({"inps": [x for x in p['inps']], "outs": [x for x in p['outs']]})

make special start and end places

In [11]:
il = {"inps": [], "outs": start_transitions}
ol = {"inps": end_transitions, "outs": []}

places.append(il)
places.append(ol)

generate input and output archs to places

In [12]:
for p in places:
  for i in p['inps']:
    archs.append({'from':i, 'to':p})

  for o in p['outs']:
    archs.append({'from':p, 'to':o})

In [13]:
import json
with open("/content/drive/MyDrive/Hospital_petrynet.json", "w") as file:
  file.write(json.dumps({'transitions': transitions, 'places': places, 'archs': archs}))